# spark sharing


In [112]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import lit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer
from pyspark.sql.functions import regexp_replace
# from pyspark.sql.functions import when
from pyspark.sql import functions as F
from pyspark.sql.functions import *

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer

import numpy as np
import pandas as pd

In [ ]:
1.basis
    1.1 SparkSession--DataFrame&SQL的入口
    1.2 DataFrame操作
    1.3 SQL的操作
    1.4 Estimator, Transformer, and Param
2.LR

3.GBDT

## 1、basis

### 1.1 SparkSession是DataFrame和SQL功能的主入口

In [52]:
# spark = SparkSession.builder.master("spark://10.0.120.106:7077").appName("spark_app").config("", "").getOrCreate()

# spark = SparkSession.builder.master("local").appName("spark_app").config("", "").getOrCreate()

# main entry of programming Spark with DataFrame. 'enableHiveSupport()' enables spark to utilize SQL or HQL
# A SparkSession can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files.
spark = SparkSession.builder.master("local").appName("spark_app").config("", "").enableHiveSupport().getOrCreate()

In [53]:
spark

### 1.2 读取数据文件

In [54]:
# 虽然是txt格式，但是按照csv来读取可以设置seperator
read_data = spark.read.csv('C:\spark\data\my_data\iris_data.txt', sep=',')
read_data.head(5)

[Row(_c0='5.1', _c1='3.5', _c2='1.4', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.9', _c1='3.0', _c2='1.4', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.7', _c1='3.2', _c2='1.3', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.6', _c1='3.1', _c2='1.5', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='5.0', _c1='3.6', _c2='1.4', _c3='0.2', _c4='Iris-setosa')]

### 1.3 拆分数据集

In [55]:
train_data, test_data = read_data.randomSplit([0.7, 0.3], 1)

In [56]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [57]:
test_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

### 1.4 操作dataframe

#### A distributed collection of data grouped into named columns. A DataFrame is equivalent to a relational table in Spark SQL

In [58]:
# show the structure of dataframe - [train_data]
train_data.show(50)

+---+---+---+---+---------------+
|_c0|_c1|_c2|_c3|            _c4|
+---+---+---+---+---------------+
|4.3|3.0|1.1|0.1|    Iris-setosa|
|4.4|2.9|1.4|0.2|    Iris-setosa|
|4.4|3.0|1.3|0.2|    Iris-setosa|
|4.4|3.2|1.3|0.2|    Iris-setosa|
|4.6|3.1|1.5|0.2|    Iris-setosa|
|4.6|3.2|1.4|0.2|    Iris-setosa|
|4.6|3.4|1.4|0.3|    Iris-setosa|
|4.6|3.6|1.0|0.2|    Iris-setosa|
|4.7|3.2|1.3|0.2|    Iris-setosa|
|4.8|3.0|1.4|0.1|    Iris-setosa|
|4.8|3.0|1.4|0.3|    Iris-setosa|
|4.8|3.1|1.6|0.2|    Iris-setosa|
|4.8|3.4|1.6|0.2|    Iris-setosa|
|4.8|3.4|1.9|0.2|    Iris-setosa|
|4.9|2.5|4.5|1.7| Iris-virginica|
|4.9|3.1|1.5|0.1|    Iris-setosa|
|4.9|3.1|1.5|0.1|    Iris-setosa|
|5.0|2.0|3.5|1.0|Iris-versicolor|
|5.0|2.3|3.3|1.0|Iris-versicolor|
|5.0|3.2|1.2|0.2|    Iris-setosa|
|5.0|3.3|1.4|0.2|    Iris-setosa|
|5.0|3.4|1.5|0.2|    Iris-setosa|
|5.0|3.5|1.3|0.3|    Iris-setosa|
|5.0|3.5|1.6|0.6|    Iris-setosa|
|5.0|3.6|1.4|0.2|    Iris-setosa|
|5.1|3.3|1.7|0.5|    Iris-setosa|
|5.1|3.4|1.5|0

In [59]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [60]:
# get all columns' names
train_data.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

In [61]:
# get the num of rows
train_data.count()

106

In [62]:
# filter
train_data.filter(train_data._c3=='0.1').show()

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.2|4.1|1.5|0.1|Iris-setosa|
+---+---+---+---+-----------+



In [63]:
# contains
train_data.filter(train_data._c4.contains('virginica')).show(5)

+---+---+---+---+--------------+
|_c0|_c1|_c2|_c3|           _c4|
+---+---+---+---+--------------+
|4.9|2.5|4.5|1.7|Iris-virginica|
|5.6|2.8|4.9|2.0|Iris-virginica|
|5.7|2.5|5.0|2.0|Iris-virginica|
|5.8|2.7|5.1|1.9|Iris-virginica|
|5.8|2.7|5.1|1.9|Iris-virginica|
+---+---+---+---+--------------+
only showing top 5 rows



In [64]:
# The available aggregate functions are avg, max, min, sum, count.
train_data.agg({'_c3': 'max'}).show()

+--------+
|max(_c3)|
+--------+
|     2.5|
+--------+



In [65]:
train_data.agg({'_c3': 'avg'}).show()

+-----------------+
|         avg(_c3)|
+-----------------+
|1.161320754716981|
+-----------------+



In [66]:
train_data.agg({'_c3': 'sum'}).show()

+------------------+
|          sum(_c3)|
+------------------+
|123.09999999999998|
+------------------+



In [67]:
# Creates or replaces a local temporary view
train_data.createOrReplaceTempView('train_table')

In [68]:
# query like sql
train_1 = spark.sql("select * from train_table where _c4=='Iris-setosa'")
train_1.show()

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.4|3.0|1.3|0.2|Iris-setosa|
|4.4|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|4.6|3.2|1.4|0.2|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|4.6|3.6|1.0|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.8|3.0|1.4|0.3|Iris-setosa|
|4.8|3.1|1.6|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.4|1.9|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.0|3.2|1.2|0.2|Iris-setosa|
|5.0|3.3|1.4|0.2|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|5.0|3.5|1.3|0.3|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [69]:
# describe the dataframe by providing basic statistics for numeric and string columns
train_data.describe().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|               _c0|                _c1|               _c2|               _c3|           _c4|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               106|                106|               106|               106|           106|
|   mean| 5.770754716981132|  3.071698113207549|3.6443396226415086| 1.161320754716981|          null|
| stddev|0.7997579373228328|0.44588162362012673|1.7494532581939912|0.7643946568198022|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+-------------------+------------------+------------------+--------------+



In [70]:
# add some quartiles statistics infomation
train_data.summary().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|               _c0|                _c1|               _c2|               _c3|           _c4|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               106|                106|               106|               106|           106|
|   mean| 5.770754716981132|  3.071698113207549|3.6443396226415086| 1.161320754716981|          null|
| stddev|0.7997579373228328|0.44588162362012673|1.7494532581939912|0.7643946568198022|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    25%|               5.1|                2.8|               1.5|               0.3|          null|
|    50%|               5.7|                3.0|               4.1|               1.3|          null|
|    75%|               6.3|                3.4|               5.1|               

In [71]:
# summary for specific column and statistics
train_data.select('_c0').summary('mean').show()

+-------+-----------------+
|summary|              _c0|
+-------+-----------------+
|   mean|5.770754716981132|
+-------+-----------------+



In [72]:
train_data.select('_c0')

DataFrame[_c0: string]

In [73]:
train_data.select('_c0').show(10)

+---+
|_c0|
+---+
|4.3|
|4.4|
|4.4|
|4.4|
|4.6|
|4.6|
|4.6|
|4.6|
|4.7|
|4.8|
+---+
only showing top 10 rows



In [74]:
train_data.select('_c0').collect()

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.7'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.9'),
 Row(_c0='4.9'),
 Row(_c0='4.9'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.2'),
 Row(_c0='5.2'),
 Row(_c0='5.3'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.8'),
 Row(_c0='5.8'),
 Row(_c0='5.8'),
 Row(_c0='5.8'

In [75]:
train_data.select('_c0').head(5)

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6')]

In [76]:
type(train_data.select('_c0').head(5))

list

In [77]:
train_data.select('_c0').take(5)

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6')]

In [78]:
train_data.select('_c0').take(5)[0]

Row(_c0='4.3')

In [79]:
type(train_data.select('_c0').take(5)[0])

pyspark.sql.types.Row

In [80]:
train_data.select('_c0').take(5)[0][0]

'4.3'

In [81]:
type(train_data.select('_c0').take(5)[0][0])

str

### 1.5 操作column

In [82]:
# pick a column in a dataframe
train_data._c0

Column<b'_c0'>

In [83]:
train_data['_c0']

Column<b'_c0'>

In [84]:
# show the result - 'column' object is not callable
train_data['_c0'].collect()

TypeError: 'Column' object is not callable

In [85]:
# do calculations on dataframe through column
operate_data = train_data.select(['_c0', '_c1'])
operate_data
operate_data.show(5)

+---+---+
|_c0|_c1|
+---+---+
|4.3|3.0|
|4.4|2.9|
|4.4|3.0|
|4.4|3.2|
|4.6|3.1|
+---+---+
only showing top 5 rows



In [86]:
# sum
add_col = operate_data._c0 + operate_data['_c1']
add_col

Column<b'(_c0 + _c1)'>

In [87]:
add_operate_data = operate_data.withColumn('sum', add_col)

In [88]:
add_operate_data.show(5)

+---+---+------------------+
|_c0|_c1|               sum|
+---+---+------------------+
|4.3|3.0|               7.3|
|4.4|2.9| 7.300000000000001|
|4.4|3.0|               7.4|
|4.4|3.2|7.6000000000000005|
|4.6|3.1| 7.699999999999999|
+---+---+------------------+
only showing top 5 rows



## 1、LR

In [93]:
train_data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.4|3.0|1.3|0.2|Iris-setosa|
|4.4|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|4.6|3.2|1.4|0.2|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|4.6|3.6|1.0|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows



In [97]:
# rename columns
train_data = train_data.withColumnRenamed('_c4', 'label')
train_data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|      label|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.4|3.0|1.3|0.2|Iris-setosa|
|4.4|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|4.6|3.2|1.4|0.2|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|4.6|3.6|1.0|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows



In [106]:
### encode the categories
train_data.select(train_data.label, F.when(train_data.label=='Iris-setosa', 1).when(train_data.label=='Iris-virginica', 2).otherwise(3)).show()

+---------------+--------------------------------------------------------------------------------------+
|          label|CASE WHEN (label = Iris-setosa) THEN 1 WHEN (label = Iris-virginica) THEN 2 ELSE 3 END|
+---------------+--------------------------------------------------------------------------------------+
|    Iris-setosa|                                                                                     1|
|    Iris-setosa|                                                                                     1|
|    Iris-setosa|                                                                                     1|
|    Iris-setosa|                                                                                     1|
|    Iris-setosa|                                                                                     1|
|    Iris-setosa|                                                                                     1|
|    Iris-setosa|                                      

In [122]:
en_label = train_data.select(F.when(train_data.label=='Iris-setosa', 1).when(train_data.label=='Iris-virginica', 2).otherwise(3))
en_label.columns

['CASE WHEN (label = Iris-setosa) THEN 1 WHEN (label = Iris-virginica) THEN 2 ELSE 3 END']

In [123]:
en_label_col = en_label['CASE WHEN (label = Iris-setosa) THEN 1 WHEN (label = Iris-virginica) THEN 2 ELSE 3 END']
en_label_col

Column<b'CASE WHEN (label = Iris-setosa) THEN 1 WHEN (label = Iris-virginica) THEN 2 ELSE 3 END'>

In [ ]:
# error ?? 
train_data_1 = train_data.withColumn('encoded label', en_label_col)

In [125]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, label: string]

## 2、GBDT

In [20]:
train_data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows



In [21]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]